In [1]:
from IPython.display import Latex

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 6)
from numba import njit, prange
import numpy as np

from wf import build_psi, V_lj, eta, epsilon, sigma

np.random.seed(0)

## Parameters starting values

In [2]:
cn = np.array([0.85, 0.1, 0.04, 0.01])

Lambda = -2.143 * sigma**2
r0 = 2.096 / sigma
w = 1.278 / sigma

In [3]:
N = 108

rho0 = 21.86e-3 * sigma**3
L = (N / (rho0))**(1/3)

psi, f, h, f0, f1, f2, f3 = build_psi(L) 

Latex(f'$L = {L:.3f}\\ \\sigma$')

<IPython.core.display.Latex object>

In [4]:
from itertools import product


def fcc_lattice(N, a):
    """
    Return a numpy array for a fcc-lattice, where the lattice is a cube of
    elementary cells. Then N must be of the type: N = 4 * n**3, where n is the 
    number of cells per side of the cubic box. "a" is the lattice parameter
    """
    lattice = np.zeros((N, 3))
    
    # first unit cell position
    unit_cell = np.array([[0,   0,   0],
                          [a/2, a/2, 0],
                          [0,   a/2, a/2],
                          [a/2, 0,   a/2]])
    
    # number of cells in the box
    n = N // 4
    
    # number of cells per side
    cells_per_side = round(n**(1/3))
    
    # coordinates of cells in the box
    cells_coordinates = product(range(cells_per_side), repeat=3)
    for i, coords in enumerate(cells_coordinates):
        
        # cell translation matrix
        R = np.zeros((4, 3))

        R[:, 0] = int(coords[0]) * a
        R[:, 1] = int(coords[1]) * a
        R[:, 2] = int(coords[2]) * a
        
        lattice[4*i:4*(i+1)] = unit_cell + R
        
    return lattice


conf = np.zeros((10000, N, 3))
conf[0] = fcc_lattice(N, L/3)

## TODO:
* sampling
* energy computation
* radial distribution function computation
* optimization technique

## ideas:
* d (healing distance) as variational parameter (now d = L / 2)

In [5]:
psi(conf[0], cn, Lambda, w, r0)

# %timeit -r 5 -n 1000 psi(conf, cn, Lambda, w, r0)

1.1347005232453426e-103

In [6]:
@njit
def equilibration(conf):
    delta = L / 100
    eq_steps = 10000

    eta = np.random.rand(eq_steps)
    to_move = np.random.choice(N, size=(eq_steps, N//3))
    move = np.random.rand(eq_steps, N//3, 3) - 0.5
    
    fw_steps = 0

    P_i = psi(conf[0], cn, Lambda, w, r0)**2
#     P_i = psi(conf, cn, Lambda, w, r0)**2
    for i in range(eq_steps-1):
        conf[i+1] = np.copy(conf[i])
        
        proposal = np.copy(conf[i])
        proposal[to_move[i]] = conf[i][to_move[i]] + delta * move[i]
        proposal = proposal - L * np.floor(proposal / L) 
        
#         proposal = np.copy(conf)
#         proposal[to_move[i]] = conf[to_move[i]] + delta * move[i]
#         proposal = proposal - L * np.floor(proposal / L)
        
        P_p = psi(proposal, cn, Lambda, w, r0)**2
        if (P_p / P_i) > eta[i]:
            conf[i+1][to_move[i]] += delta * move[i]
#             conf[to_move[i]] += delta * move[i]
            P_i = float(P_p)
            fw_steps += 1
        
        if ((i+1) % 500 == 0):
            print(i+1, delta, fw_steps)
            if (fw_steps > 350) or (fw_steps < 150):
                delta = delta * (0.5 + fw_steps/500)
            # reset forward steps counter
            fw_steps = 0
            
equilibration(conf)

In [7]:
%timeit -r 5 -n 10 equilibration(conf)

6.9 s ± 128 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
